In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import math
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import GridSearchCV

In [ ]:
words = np.zeros((10,10,100))

In [ ]:
def NonLinearFunction(prob, confidence, sharpness=10):
    return 2 / (1 + math.exp(-sharpness * (prob - confidence)))

def AggregateFunction(scores):
    product = 1.0
    for s in scores:
        product *= max(s, 1e-6)
    return product ** (1.0 / len(scores)) if scores else 0

def penalty_phenom():
    return 

In [ ]:
class SpeechClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, p_threshold=0.5, w_trehshold=0.5, confidence=0.5, sharpeness=1.0):
        self.p_threshold = p_threshold
        self.w_trehshold = w_trehshold
        self.confidence = confidence
        self.sharpeness = sharpeness

    def fit(self, X, y):
        self.words_ = X
        self.targets_ = y
        return self

    def predict(self, X):
        predictions = []
        for target, words, deletion in zip(self.targets_, X):
            result = self.classifier(target, words, deletion)
            predictions.append(int(result))
        return np.array(predictions)

    def classifier(self, target, words, deletion=None):
        S_t = target.shape[0]
        for w in words:
            S_w = w.shape[0]
            for i in range(S_w - S_t + 1):
                Scores = []
                for pw, pt in zip(w[i:i + S_t], target):
                    if deletion is not None and i!=0:
                        if np.argmax(w[i-1]) == deletion:
                            break
                    s = 0
                    prob = pw[pt]
                    if prob > self.confidence:
                        s = 1
                    else:
                        s = NonLinearFunction(prob, sharpeness=self.sharpeness, confidence=self.confidence)

                    Scores.append(s)

                if AggregateFunction(Scores) > self.w_trehshold:
                    for score in Scores:
                        if score < self.p_threshold:
                            break
                        return True
        return False

In [ ]:
param_grid = {
    'p_threshold': [0.4, 0.5, 0.6],
    'w_trehshold': [0.3, 0.5, 0.7],
    'confidence': [0.4, 0.5, 0.6],
    'sharpeness': [1.0, 2.0, 5.0]
}

grid = GridSearchCV(SpeechClassifier(), param_grid, scoring='precision', cv=4)
grid.fit(X_data, y_labels)